# Effect of frequency dependent conductivity on extracellular potentials

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack as ff
from brainsignals.plotting_convention import mark_subplots, simplify_axes

np.random.seed(1234)

def conductivity_from_exp_data(freqs):
    from scipy.interpolate import interp1d

    # Ranck (1963)
    exp_freqs = [5, 5000]
    exp_sigmas = [1./3.56, 1./2.56]
    
    # Need to interpolate so data is on same frequencies as FFT og signal
    if exp_freqs[-1] < freqs[-1]:
        exp_freqs = np.r_[exp_freqs, freqs[-1]]
        exp_sigmas = np.r_[exp_sigmas, exp_sigmas[-1]]

    if exp_freqs[0] > freqs[0]:
        exp_freqs = np.r_[freqs[0], exp_freqs]
        exp_sigmas = np.r_[exp_sigmas[0], exp_sigmas]

    f = interp1d(exp_freqs, exp_sigmas, kind='linear')
    interp_sig = f(freqs)

    sigma_freqs = interp_sig
    return sigma_freqs

def return_phase_from_modulus(modulus_f):
    # Based on Clark et al. (2005)
    # https://doi.org/10.2172/15014444
    N = len(modulus_f)
    odd = N % 2
    aa = int((N+odd)/2 - 1)
    bb = int((N-odd)/2 - 1)
    u_plus = np.r_[1, 2*np.ones(aa), 1, np.zeros(bb)]
    cn = np.real(ff.ifft(np.log(modulus_f)))
    cf = u_plus * cn
    theta = np.imag(ff.fft(cf))
    return theta



In [ ]:
T = 100
time_res = 2**-7

num_tsteps = int(T / time_res + 1)
tvec = np.arange(num_tsteps) * time_res

sample_freq = ff.fftfreq(num_tsteps, d=time_res/1000.)
pidxs = np.where(sample_freq >= 0)
freqs = sample_freq[pidxs]

avrg_sigma = 0.29

sigma_fixed = avrg_sigma * np.ones(len(freqs))
sigma_ranck = conductivity_from_exp_data(freqs)

sigma_fixed_ = np.r_[sigma_fixed, sigma_fixed[::-1][1:]]
sigma_ranck_ = np.r_[sigma_ranck, sigma_ranck[::-1][1:]]

phase_fixed_ = return_phase_from_modulus(1./sigma_fixed_)
phase_ranck_ = return_phase_from_modulus(1./sigma_ranck_)

point_source_pos = np.array([0, 0, 0])
elec_pos = np.array([10, 0, 0])
dist = np.sqrt(np.sum((point_source_pos - elec_pos)**2))

imem_step = np.zeros(num_tsteps)
t0, t1 = 5, 6
t0_idx = np.argmin(np.abs(tvec - t0))
t1_idx = np.argmin(np.abs(tvec - t1))
imem_step[t0_idx:t1_idx] = 50.

imem_noise = np.sum([np.sin(2 * np.pi * f * tvec / 1000 + 2 * np.pi *
                      np.random.uniform(0, 1))
               for f in range(1, 1000)], axis=0)

sig_step = 1./(4 * np.pi * 1) * dist * imem_step
sig_noise = 1./(4 * np.pi * 1) * dist * imem_noise

T_ranck = np.exp(1j * phase_ranck_) /sigma_ranck_
T_fixed = np.exp(1j * phase_fixed_) /sigma_fixed_

Y_step = ff.fft(sig_step)
Y_noise = ff.fft(sig_noise)

sig_fixed_step = np.real(ff.ifft(Y_step * T_fixed))
sig_fixed_noise = np.real(ff.ifft(Y_noise * T_fixed))
sig_ranck_step = np.real(ff.ifft(Y_step * T_ranck))
sig_ranck_noise = np.real(ff.ifft(Y_noise * T_ranck))

plt.close('all')

fig = plt.figure()
fig.subplots_adjust(left=0.08, wspace=0.8, bottom=0.25,
                    top=0.9, right=0.98)


ax0 = fig.add_subplot(141, xlim=[1, 1000],
                     ylim=[0.2, 0.4], yticks=[0.2, 0.3, 0.4],
                     xticks=[0, 500, 1000],
                     title="conductivity",
                     xlabel='frequency (Hz)',
                     ylabel='|$\sigma_{\mathrm{t}}$| (S/m)')

ax_dict = dict(xlabel="time (ms)", xlim=[0, 10], ylabel="V$_e$ (µV)")

ax1 = fig.add_subplot(142, xlim=[1, 1000],
                     xticks=[0, 500, 1000], ylim=[-30, 30],
                        yticks=[-30, -15, 0, 15, 30],
                     title="phase shift",
                     xlabel='frequency (Hz)',
                     ylabel='degrees')

ax2 = fig.add_subplot(143, title="noisy signal", **ax_dict)
ax3 = fig.add_subplot(144, title="step pulse", **ax_dict)

fixed_clr = 'r'
wagner_clr = 'k'
l0, = ax0.plot(freqs, sigma_fixed, lw=2, color=fixed_clr)
l1, = ax0.plot(freqs, sigma_ranck, ls='-', lw=2, color=wagner_clr)

ax1.plot(freqs, np.rad2deg(phase_fixed_[pidxs]), lw=2, color=fixed_clr)
ax1.plot(freqs, np.rad2deg(phase_ranck_[pidxs]), lw=2, ls='-', color=wagner_clr)

lines = [l0, l1]
line_names = ["constant", "Ranck (1963)"]

ax2.plot(tvec, sig_fixed_noise, color=fixed_clr, lw=2)
ax2.plot(tvec, sig_ranck_noise, color=wagner_clr, lw=1.5, ls='-',)
ax3.plot(tvec, sig_fixed_step, color=fixed_clr, lw=2)
ax3.plot(tvec, sig_ranck_step, color=wagner_clr, lw=1.5, ls='-',)

fig.legend(lines, line_names, loc=(0.45, 0.01), ncol=2, frameon=False)
mark_subplots([ax0, ax1, ax2, ax3], xpos=-0.21, ypos=1.1)
simplify_axes([ax0, ax1, ax2, ax3])

fig.savefig('effect_of_freq_dependence.pdf')
